# Run DESYNC-MHP MLE on a toy example

This notebook shows an example of the DESYNC-MLE algorithm on a small toy network.
The toy example is the one used in the paper and is defined as follows: 

We generate a synthetic realization from a multivariate Hawkes process with exponential excitation functions in $d=2$ dimensions (i.e., processes $N_1$ and $N_2$). Process $N_1$ excites itself and influences process $N_2$ (but not the other way around). In other words, the excitation matrix is 

\begin{align}
    A = 
    \begin{bmatrix} 
    \alpha_{11}  e^{-\beta t} 1_{\{t > 0\}} & 0 \\
    \alpha_{21}  e^{-\beta t} 1_{\{t > 0\}} & 0 \\
    \end{bmatrix}  
\end{align}

Therefore, the intensity of the processes are
\begin{align}
 \lambda_1(t) = \mu_1 + \alpha_{11} e^{-\beta t} 1_{\{t > 0\}}  + \alpha_{21} e^{-\beta t} 1_{\{t > 0\}} 
 \quad \text{and} \quad
 \lambda_2(t) = \mu_2.
\end{align}

As discussed in the paper, for a synchronization noise such that $z_1 > z_2$, the cause and effect arrivals start to blur and the classic maxmimum likelihood estimation converges to a complete graph. However, our approach, DESYNC-MHP MLE, is able to accurately recover the true causal structure of the excitation matrix.

---

Load libraries

In [ ]:
import sys
if '..' not in sys.path:
    # Allows to import the `lib` module holding the code for DESYNC-MHP MLE
    sys.path.append('..')

# External libs
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx

# Use `tick` to simulate synthetic data and for the baseline `Classic MLE` estimator
from tick.hawkes.simulation import SimuHawkesExpKernels
from tick.hawkes.inference import HawkesADM4

# Code for the `DESYNC-MLE` algorithm
from lib.inference.learner_conditional_mle import HawkesExpKernConditionalMLE

%matplotlib inline

# Graph drawing parameters
nx_draw_params = {
    'pos':{0: (0,0), 1: (1, 0)}, 
    'node_size': 2e3, 'node_color': 'w', 
    'labels': {0: '$N_1$', 1: '$N_2$'},
    'edgecolors':'k', 'arrowsize': 50, 'width': 2,
    'font_size': 20
}

Initialize random seed.

In [ ]:
np.random.seed(12345678)

---

## 1. Define parameters

Set parameters for the simulation of synthetic data. 

* `n_nodes` is the number of nodes of the process,
* `decay` is the exponential decay $\beta$ of the kernels,
* `end_time` is the length of the observation window,
* `noise_scale` is the variance used to sample the synchronization noise
* `baseline` is the vector of background intensity $\boldsymbol{\mu} = [\mu_1, \mu_2]$,
* `adjacency` is the matrix of magnitudes $\{\alpha_{ij}\}$ of the kernels.

Feel free to try the code with other parameters (*e.g.* larger network, increasing the noise scale, ...).

In [ ]:
n_nodes = 2         # Number of nodes
decay = 1.0         # Exponential decay
end_time = 100e3    # Length of the observation window
noise_scale = 1.0   # Scale (variance) of the noise 

# Background intensity
mu = 0.05
baseline = mu * np.ones(n_nodes, dtype='float')

# Excitation matrix
adjacency = np.array([[0.9, 0.0],
                      [0.9, 0.0]])

In [ ]:
print('True baseline:')
print(baseline.round(2))
print('True adjacency:')
print(adjacency.round(2))

# Draw the graph
print('\nEstimated excitation graph:')
graph_true = nx.DiGraph(adjacency.T)
plt.figure(figsize=(8, 1))
nx.draw_networkx(graph_true, **nx_draw_params)
plt.axis('off');

---

## 2. Simulate data

Sample a random synchronization noise value for each dimension.

In [ ]:
z_true = np.random.normal(scale=noise_scale, size=n_nodes)

# Order the noise value to be in the "bad" regime 
# (where the events from N_1 are delayed more than those from N_2)
z_true = np.sort(z_true)[::-1]

z_true

Sample a realization of the multivariate Hawkes process with exponential excitation kernels.

In [ ]:
# Compute the observation window adjusted to the noise to avoid edge effects
adjusted_end_time = end_time + z_true.min() - z_true.max()
adjusted_start_time = z_true.max()

seed = 42

simu = SimuHawkesExpKernels(adjacency=adjacency, decays=decay, baseline=baseline,
                            end_time=adjusted_end_time, seed=42)
simu.simulate()

print('\nNumber of events simulated:', list(map(len, simu.timestamps)))

Add synchronization noise to the observed events.

In [ ]:
# Hold the noisy realizations
noisy_events = []

# For each dimensions
for m, orig_events_m in enumerate(simu.timestamps):
    # Add the noise
    events_m = orig_events_m + z_true[m] - adjusted_start_time
    # Filter the events outside the observation window to avoid edge effects
    events_m = events_m[(events_m >= 0) & (events_m < end_time)]
    noisy_events.append(events_m)

Visualize the observations.

In [ ]:
print('Original noiseless events:')
for m, orig_events_m in enumerate(simu.timestamps):
    print(f'{m:>d} | {orig_events_m}')

print(f'\nNoise value: {z_true}\n')
    
print('Noisy events:')
for m, events_m in enumerate(noisy_events):
    print(f'{m:>d} | {events_m}')

---

## 3. Inference

Set the starting parameters values at random.

In [ ]:
z_start = np.random.normal(scale=noise_scale, size=n_nodes)

baseline_start = np.random.uniform(0.0, 1.0, size=n_nodes)
adjacency_start = np.random.uniform(0.0, 1.0, size=(n_nodes, n_nodes)) 

# Stack all parameters into a single vector
theta_start = np.hstack((baseline_start, adjacency_start.ravel()))

---

###  3.1. Classic MLE (baseline method)

In [ ]:
naive_learner = HawkesADM4(decay, lasso_nuclear_ratio=1.0,
                           verbose=True, print_every=10, record_every=10)
naive_learner.fit(noisy_events, end_time,
                  baseline_start=baseline_start,
                  adjacency_start=adjacency_start);

Visualize the results.

In [ ]:
print('\nEstimated baseline:')
print(naive_learner.baseline.round(2))
print('Ground truth baseline:')
print(baseline.round(2))

print('\nEstimated excitation matrix:')
print(naive_learner.adjacency.round(2))
print('Ground truth excitation matrix:')
print(adjacency.round(2))

adj_err = (naive_learner.adjacency > 0.05).astype(int) - (adjacency > 0).astype(int)
print('\nErrors in the excitation matrix: ({:d} errors total) (FP: 1, FN: -1)'.format(np.abs(adj_err).sum()))
print(adj_err)

print('\nEstimated excitation graph:')
plt.figure(figsize=(8, 1))
naive_learner.adjacency[naive_learner.adjacency < 0.05] = 0
graph_classic_mle = nx.DiGraph(naive_learner.adjacency.T)
nx.draw_networkx(graph_classic_mle, **nx_draw_params)
plt.axis('off');

---

### 3.2. Run DESYNC-MHP MLE (contribution)

Initialize the learner object.

In [ ]:
# Define the step size
step_size_func = lambda t: 3.0 * (t+1) ** -0.5

learner = HawkesExpKernConditionalMLE(decay,
    approx_type='smooth',      # Use the smooth approximation of the exponential kernel
    decay_neg=50.0,            # Negative decay of the smooth kernel
    gamma=100.0,               # Transition rate between negative/positive parts of the smooth kernel
    n_threads=2,               # Number of threads used for computations
    max_iter=1000,             # Maximum number of iterations
    step_z=step_size_func,     # Step size
    step_theta=step_size_func, # Step size
)

Run the optimization.

In [ ]:
coeffs = learner.fit(
    noisy_events, end_time,   # Set observed data
    z_start=z_start,          # Initial noise parameters value
    theta_start=theta_start,  # Initial MHP parameters
    seed=42,                  # Set random seed for SGD
)

Visualize the results.

In [ ]:
print('\nEstimated baseline:')
print(learner.baseline.round(2))
print('Ground truth baseline:')
print(baseline.round(2))

print('\nEstimated excitation matrix:')
print(learner.adjacency.round(2))
print('Ground truth excitation matrix:')
print(adjacency.round(2))

adj_err = (learner.adjacency > 0.05).astype(int) - (adjacency > 0).astype(int)
print('\nErrors in the excitation matrix: ({:d} errors total) (FP: 1, FN: -1)'.format(np.abs(adj_err).sum()))
print(adj_err)

print('\nEstimated excitation graph:')
plt.figure(figsize=(8, 1))
learner.adjacency[learner.adjacency < 0.05] = 0
graph = nx.DiGraph(learner.adjacency.T)
nx.draw_networkx(graph, **nx_draw_params)
plt.axis('off');